<a href="https://colab.research.google.com/github/evroth/Congressional_Analytics/blob/main/Congressional_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This document contais the content for our first project in the Cloud Computing Class.
The analysis below goes attempts to look at the number of starbucks locations in each congressional district, then look at whether the district is red or blue.
We will gather data for Starbucks locations off of Kaggle. Census data from Census.gov for Total Population, District Number, and State. Lastly data from some source on Political Affiliation, State, and District Number. We will do some EDA and merge the datasets together to perform our analysis.

In [2]:
# starbucks locations
## I am not sure how to always have this data downloaded, might need to re-upload it to colab every time?
import pandas as pd
from geopy.geocoders import Nominatim

# read store location data into a dataframe
store_df = pd.read_csv("starbucks_locations.csv")

# select only US stores
store_df = store_df[store_df['Country'] == "US"]

# Select the "Store Number", "State/Province", "Longitude", and "Latitude" columns
store_df = store_df[["Store Number", "State/Province", "Longitude", "Latitude"]]

# print the dataframe
print(store_df)

       Store Number State/Province  Longitude  Latitude
11964   3513-125945             AK    -149.78     61.21
11965   74352-84449             AK    -149.84     61.14
11966  12449-152385             AK    -149.85     61.11
11967  24936-233524             AK    -149.89     61.13
11968    8973-85630             AK    -149.86     61.14
...             ...            ...        ...       ...
25567   74385-87621             WY    -105.59     41.32
25568   73320-24375             WY    -105.56     41.31
25569  22425-219024             WY    -105.56     41.31
25570  10849-103163             WY    -109.25     41.58
25571  10769-102454             WY    -106.94     44.77

[13608 rows x 4 columns]


In [11]:
# Install Library
python3 -m pip install --upgrade pip

SyntaxError: ignored

In [18]:
# Convert Lat and Long to Congressional Districts
import geopandas as gpd

# convert the store location data into a geodataframe
store_gdf = gpd.GeoDataFrame(store_df, geometry=gpd.points_from_xy(store_df.Longitude, store_df.Latitude))

# read congressional district data into a geodataframe
district_gdf = gpd.read_file("congressional_districts.shp")

# merge the store location data with the congressional district data
merged_gdf = gpd.sjoin(store_gdf, district_gdf, op='within')

# print the merged data
print(merged_gdf)

ModuleNotFoundError: ignored

In [16]:
import requests

# add new column for the district
store_df["District"] = None

# API endpoint
url = "https://geo.fcc.gov/api/census/block/find"

# loop through the rows of the dataframe
for i, row in store_df.iterrows():
    # get the latitude and longitude
    latitude = row["Latitude"]
    longitude = row["Longitude"]

    # API parameters
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "showall": False,
        "format": "json"
    }

    # Send the request
    response = requests.get(url, params=params)

    # Get the district number from the response
    district = response.json()["Congressional District Number"]
    #### I can't figure out what the name to pull is from the API

    # add the district number to the District column
    store_df.at[i, "District"] = district

# print the dataframe
print(store_df)


KeyError: ignored

In [17]:
import requests

# add new column for the district
store_df["District"] = None

# API endpoint
url = "https://congress.api.sunlightfoundation.com"

# loop through the rows of the dataframe
for i, row in store_df.iterrows():
    # get the latitude and longitude
    latitude = row["Latitude"]
    longitude = row["Longitude"]

    # API parameters
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "showall": False,
        "format": "json"
    }

    # Send the request
    response = requests.get(url, params=params)

    # Get the district number from the response
    district = response.json()["District"]
    state = response.json()["State"]

    # add the district number to the District column
    store_df.at[i, "District"] = district
    

# print the dataframe
print(store_df)

SSLError: ignored